In [4]:
import pandas as pd
import numpy as np
import idx2numpy

In [6]:
train_images = idx2numpy.convert_from_file('train-images-idx3-ubyte/train-images-idx3-ubyte')
train_labels = idx2numpy.convert_from_file('train-labels-idx1-ubyte/train-labels-idx1-ubyte')
t10k_labels = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')
t10k_images = idx2numpy.convert_from_file('t10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
